# ShopRun and ShopCase

1. Explore ShopRun
2. Create ShopCase
3. Trigger ShopRun from ShopCase
4. Edit Case File
5. Edit Case Data in Python

## Preparation

Instantiating `PowerOpsClient`...

In [ ]:
# You can control which setting files are loaded through the environmental variable below. 
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops import PowerOpsClient
os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

## Explore ShopRun

`ShopRun` represents a single attempt at running SHOP.

In [ ]:
# Retrieve an existing shop run by event external_id
shop_run_0 = powerops.shop.retrieve("POWEROPS_SHOP_RUN_2023-10-26T09:56:27.964670Z_efc805")
print(shop_run_0)

In [ ]:
shop_run_0.check_status()

In [ ]:
list(shop_run_0.get_shop_files())
# also available:
# shop_run_0.get_case_file()
# list(shop_run_0.get_log_files())

## Create ShopCase

`ShopCase` is an editable representation of a SHOP case. It is used to prepare data for a SHOP run.

We run SHOP by "triggering" `ShopCase`. This creates a new `ShopRun` instance.

`ShopCase` can be created from an existing `ShopRun` instance (using `shop_run.as_case()`) or by directly instantiating the class. The former is probably useful more often.

In [ ]:
# Re-run SHOP using the same data 
case_1 = shop_run_0.to_case()
shop_run_1 = powerops.shop.trigger_case(case_1)
shop_run_1.check_status()

In [ ]:
# A small helper function to wait for a shop run to finish
from time import sleep
def wait(shop_run):
    while shop_run.check_status() == "in_progress":
        sleep(1)
        print(".", end="")
    print("DONE")
    
wait(shop_run_1)
shop_run_1.check_status()

# Edit Case File

We will save the case YAML file to local filesystem, then make some changes to it and use the changed file to run SHOP again. 

The case file is here edited using Python, but in real use it is expected that the file be edited using external tools.

In [ ]:
# Make a new ShopCase and save the case file to local filesystem
case = shop_run_1.to_case()
case.save_yaml("case_file.yaml")

# Shop top of the file:
def shop_top_of_file(file_name):
    print(open(file_name).read(200) + "...")
    
shop_top_of_file("case_file.yaml")

In [ ]:
# Deliberately break the YAML structure of the file. 
# Here we delete first 15 lines of the file for demonstration purposes.
with open("case_file.yaml") as case_file:
    lines = case_file.readlines()
with open("case_file.yaml", "w") as case_file:
    case_file.writelines(lines[15:])
shop_top_of_file("case_file.yaml")

In [ ]:
# This is now an invalid YAML file, so we cannot use it with ShopCase.
try:
    case.load_case_file("case_file.yaml")
except Exception as exc:
    print(repr(exc))

In [ ]:
# Add "commands:" line to the beginning of the file.
# This will fix the YAML structure, but the case itself will remain broken.
broken_case = open("case_file.yaml").read()
with open("case_file.yaml", "w") as case_file:
    case_file.write("commands:\n")
    case_file.write(broken_case)
shop_top_of_file("case_file.yaml")

In [ ]:
# Now we have a technically valid YAML, we can use it to run SHOP.
case.load_case_file("case_file.yaml")
shop_run_2 = powerops.shop.trigger_case(case)
shop_run_2.check_status()

In [ ]:
# We expect SHOP run to run and fail, though.
wait(shop_run_2)
shop_run_2.check_status()

## Edit data in Python

Besides editing the case file and using `case.load_case_file()`, we can also interact with case data directly, using `case.data` dict.

We will fix the case data by adding the missing commands back.

In [ ]:
# Add missing commmands back, direclty into case data.
case.data["commands"] = [
     'set time_delay_unit MINUTE',
     'set ramping /on',
     'set stop_cost_from_start_cost /on',
     'set bypass_loss /on',
     'set mipgap 0.001000',
     'set timelimit 600.000',
     'set reserve_ramping_cost 1',
     'set fcr_n_equality /on',
     'set reserve_slack_cost 1',
     'set reserve_min_capacity 0.02',
     'set dyn_seg /on',
     'set dyn_juncloss /on',
     'penalty flag /on /plant /schedule',
     'start sim 3',
     'set code /inc',
     'start sim 4',
]

In [ ]:
# This time we expect the SHOP run to succeed.
shop_run_3 = powerops.shop.trigger_case(case)
wait(shop_run_3)
shop_run_3.check_status()

## Cleanup

In [ ]:
# remove the local case file
import os
os.unlink("case_file.yaml")

End of notebook.